# 12.1. Stress Field Problem: Generate Data Repository

In [ ]:
#@title 12.1.1. Import some necessary packages
import numpy as np
import matplotlib.pyplot as plt

# to avoide typing np., we gather some necessary functions as follows:
cos  = np.cos 
sin  = np.sin
pi   = np.pi
sqrt = np.sqrt
acos = np.arccos

> <img src=	"	https://i.ibb.co/TWPyMrm/01.png	"	width="350"/>

In [ ]:
#@title 12.1.2. Define some problem parameters
d_star               = 3*.249e-9; 
cell_res             = 5e-6;
cell_radius          = (np.sqrt(2)/2)*cell_res;
data_num             = 10000000;

sigxx_original_lb = -1.893190846550328e+09;
sigxx_original_ub = +1.893190846550328e+09;

sigyy_original_lb = -1.893190846550328e+09;
sigyy_original_ub = +1.893190846550328e+09;

sigxy_original_lb = -1.338688085676038e+09;
sigxy_original_ub = +1.338688085676038e+09;

> <img src=	"	https://i.ibb.co/5nPFc59/02.png	"	width="750"/>

In [ ]:
#@title 12.1.3 Predefined functions: local stress in PCS 

def fun_slxx(a,r):
    slxx = -(3*sin(a)-2*sin(a)**3)/r
    return slxx

def fun_slxy(a,r):
    slxy = -(1*cos(a)-2*cos(a)**3)/r
    return slxy

def fun_slyy(a,r):
    slyy = +(1*sin(a)-2*sin(a)**3)/r
    return slyy

> <img src=	"	https://i.ibb.co/C6LQtWy/03.png	"	width="750"/>

In [ ]:
#@title 12.1.4 Predefined functions: global stress in PCS 

def fun_sgxx(a,r,t):
    sgxx = fun_slxx(a,r)*cos(t)**2+1*fun_slyy(a,r)*sin(t)**2-2*fun_slxy(a,r)*sin(t)*cos(t)
    return sgxx

def fun_sgxy(a,r,t):
    sgxy = -(fun_slyy(a,r)-fun_slxx(a,r))*sin(t)*cos(t)+(fun_slxy(a,r))*(cos(t)**2-sin(t)**2)
    return sgxy

def fun_sgyy(a,r,t):
    sgyy = 1*fun_slxx(a,r)*sin(t)**2+1*fun_slyy(a,r)*cos(t)**2+2*fun_slxy(a,r)*sin(t)*cos(t)
    return sgyy

> <img src=	"	https://i.ibb.co/kJSzXYv/12.png	"	width="1500"/>

In [ ]:
#@title 12.1.5 Predefined functions: inverse functions with respect to radials
def fun_sgxxi(a,sgxx,t):
    rlj = (sin(t)**2*(sin(a) - 2*sin(a)**3) - cos(t)**2*(3*sin(a) - 2*sin(a)**3) + 2*cos(t)*sin(t)*(cos(a) - 2*cos(a)**3))/sgxx
    return rlj

def fun_sgxyi(a,sgxy,t):
    rlj = -((cos(t)**2 - sin(t)**2)*(cos(a) - 2*cos(a)**3) + cos(t)*sin(t)*(4*sin(a) - 4*sin(a)**3))/sgxy
    return rlj

def fun_sgyyi(a,sgyy,t):
    rlj = -(sin(t)**2*(3*sin(a) - 2*sin(a)**3) - cos(t)**2*(sin(a) - 2*sin(a)**3) + 2*cos(t)*sin(t)*(cos(a) - 2*cos(a)**3))/sgyy
    return rlj

In [ ]:
#@title 12.1.6 Predefined functions: conversion, scale, and wrap
def fun_ccs2pcs(x,y):
  r = sqrt(x**2 + y**2)
  a = fun_wrap2pi(np.arctan2(y,x))

  return a,r

def fun_pcs2ccs(a,r):
    x = r*cos(a)
    y = r*sin(a)
    return x,y

def fun_wrapTo2pi(x):
  xwrap = np.remainder(x, 2*pi)
  idx = np.abs(xwrap) > 2*pi
  xwrap[idx] -= 2*pi * np.sign(xwrap[idx]);
  return xwrap

def fun_glb2loc(agb,rgb,tgb,agt,rgt):
    xgt, ygt   = fun_pcs2ccs(agt,rgt)
    xgb, ygb   = fun_pcs2ccs(agb,rgb)
    xrt        = xgt - xgb
    yrt        = ygt - ygb
    xlt        = +xrt*cos(tgb) + yrt*sin(tgb) 
    ylt        = -xrt*sin(tgb) + yrt*cos(tgb)
    alt, rlt   = fun_ccs2pcs(xlt,ylt)
    return alt,rlt

def fun_scale(x,lb=0,ub=1):
    return ((x-np.min(x,axis=0))/(np.max(x,axis=0)-np.min(x,axis=0)))*(ub-lb) + lb

def fun_scaleback(x,lb,ub):
    delta_val = ub-lb
    return x*delta_val + lb

> <img src=	"	https://i.ibb.co/nMzg5w4/05.png	"	width="350"/>

> <img src=	"	https://i.ibb.co/4FSPSgc/08.png	"	width="350"/>

> <img src=	"	https://i.ibb.co/8sVY0w6/07.png	"	width="750"/>

In [ ]:
#@title 12.1.7 Generate unbiased data repository
np.random.seed(30)

# gnerate random data in local PCS
thexx_original_i = np.random.rand(data_num,1)*2*np.pi
theyy_original_i = np.random.rand(data_num,1)*2*np.pi
thexy_original_i = np.random.rand(data_num,1)*2*np.pi

alpxx_original_j = np.random.rand(data_num,1)*2*np.pi
alpyy_original_j = np.random.rand(data_num,1)*2*np.pi
alpxy_original_j = np.random.rand(data_num,1)*2*np.pi

# generate stresses randomly
sigxx_original_j = fun_scale(np.random.rand(data_num,1), sigxx_original_lb, sigxx_original_ub)
sigyy_original_j = fun_scale(np.random.rand(data_num,1), sigyy_original_lb, sigyy_original_ub)
sigxy_original_j = fun_scale(np.random.rand(data_num,1), sigxy_original_lb, sigxy_original_ub)

# generate radials using inverse stress function in local PCS
radxx_original_j = fun_sgxxi(alpxx_original_j, sigxx_original_j, thexx_original_i)
radyy_original_j = fun_sgyyi(alpxx_original_j, sigyy_original_j, thexx_original_i)
radxy_original_j = fun_sgxyi(alpxx_original_j, sigxy_original_j, thexx_original_i)

# wrap negetive radials (if any)
ind_negetive                   = radxx_original_j < 0
alpxx_original_j[ind_negetive] = fun_wrapTo2pi(alpxx_original_j[ind_negetive] + np.pi)
radxx_original_j[ind_negetive] = np.abs(radxx_original_j[ind_negetive])

ind_negetive                   = radyy_original_j < 0
alpyy_original_j[ind_negetive] = fun_wrapTo2pi(alpyy_original_j[ind_negetive] + np.pi)
radyy_original_j[ind_negetive] = np.abs(radyy_original_j[ind_negetive])

ind_negetive                   = radxy_original_j < 0
alpxy_original_j[ind_negetive] = fun_wrapTo2pi(alpxy_original_j[ind_negetive] + np.pi)
radxy_original_j[ind_negetive] = np.abs(radxy_original_j[ind_negetive])

# control the d_start (or d) condition
ind_xx = radxx_original_j >= d_star
ind_yy = radyy_original_j >= d_star
ind_xy = radxy_original_j >= d_star

# if one ind element is false then that element is false for every plane
# combine indices and get ind_gd
ind_gd = ind_xx & ind_yy & ind_xy

# turn vectors to column vectors and only keep the datapoints with rj greater than d_star (or d)
alpxx_original_j = np.expand_dims(alpxx_original_j[ind_gd], axis = 1)
alpyy_original_j = np.expand_dims(alpyy_original_j[ind_gd], axis = 1)
alpxy_original_j = np.expand_dims(alpxy_original_j[ind_gd], axis = 1)

thexx_original_i = np.expand_dims(thexx_original_i[ind_gd], axis = 1)
theyy_original_i = np.expand_dims(theyy_original_i[ind_gd], axis = 1)
thexy_original_i = np.expand_dims(thexy_original_i[ind_gd], axis = 1)

radxx_original_j = np.expand_dims(radxx_original_j[ind_gd], axis = 1)
radyy_original_j = np.expand_dims(radyy_original_j[ind_gd], axis = 1)
radxy_original_j = np.expand_dims(radxy_original_j[ind_gd], axis = 1)

# concatenate datapoints of different planes across axis = 0 to get more datapoints 
# the rotations, azimuths, and raidals are OK and meet the constraints, no matter in what plain
alpha_original_j = np.concatenate((alpxx_original_j, alpyy_original_j, alpxy_original_j), axis = 0)
radia_original_j = np.concatenate((radxx_original_j, radyy_original_j, radxy_original_j), axis = 0)
theta_original_i = np.concatenate((thexx_original_i, theyy_original_i, thexy_original_i), axis = 0)

# use standard logarithm for radials to get a better distribution
radia_logarith_j = np.log10(radia_original_j)

# compute unbiased sigmas
sigxx_ubiasedd_i = fun_sgxx(alpha_original_j, radia_original_j, theta_original_i)
sigyy_ubiasedd_i = fun_sgyy(alpha_original_j, radia_original_j, theta_original_i)
sigxy_ubiasedd_i = fun_sgxy(alpha_original_j, radia_original_j, theta_original_i)

data_num         = radia_original_j.shape[0] # reduced 

In [ ]:
#@title 12.1.8 Make some figures
fig, axs = plt.subplots(3, 2,figsize=(10,12))

axs[0, 0].hist(theta_original_i,100);
axs[0, 0].set_title('theta_original_i');

axs[1, 0].hist(alpha_original_j,100);
axs[1, 0].set_title('alpha_original_j');

axs[2, 0].hist(radia_logarith_j ,100);
axs[2, 0].set_title('radia_logarith_j ');

axs[0, 1].hist(sigxx_ubiasedd_i,100);
axs[0, 1].set_title('sigxx_ubiasedd_i');

axs[1, 1].hist(sigyy_ubiasedd_i,100);
axs[1, 1].set_title('sigyy_ubiasedd_i');

axs[2, 1].hist(sigxy_ubiasedd_i ,100);
axs[2, 1].set_title('sigxy_ubiasedd_i');

filename = 'stress_histograms.png'
plt.savefig(filename, dpi=600)


> <img src=	"	https://i.ibb.co/nMzg5w4/05.png	"	width="350"/>

> <img src=	"	https://i.ibb.co/4FSPSgc/08.png	"	width="350"/>

In [ ]:
#@title 12.1.9 Create ML datapoints
# calibrate the data
datain_calibrated        = np.concatenate((theta_original_i/(2*np.pi), alpha_original_j/(2*np.pi), 1 - radia_logarith_j/np.log10(d_star)), axis = 1)
dataou_calibrated        = np.concatenate((sigxx_ubiasedd_i/sigxx_original_ub, sigyy_ubiasedd_i/sigyy_original_ub, sigxy_ubiasedd_i/sigxy_original_ub), axis = 1)

# # check if there is any repetitions
# datain_calibrated, index = np.unique(datain_calibrated,axis = 0, return_index=True)
# dataou_calibrated        = datain_calibrated[index,:]

# 12.2. Stress Field Problem: Machine Learning Model

> <img src=	"	https://i.ibb.co/HGnXjnH/09.png	"	width="300"/>

In [ ]:
#@title 12.2.1. Import some necessary packages
import tensorflow as tf
from tensorflow.keras.layers import Activation, Dense, Dropout
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras import regularizers
from sklearn.model_selection import train_test_split


In [ ]:
#@title 12.2.2. Train and test split
RTT = 0.1
RRS = 1

datain_tr_calibrated, datain_te_calibrated, dataou_tr_calibrated, dataou_te_calibrated = train_test_split(datain_calibrated, dataou_calibrated, test_size = RTT, random_state = 42)

In [ ]:
#@title 12.2.3. Define neural network parameters
in_neurons = datain_tr_calibrated.shape[1]
ou_neurons = dataou_tr_calibrated.shape[1]

val_split  = 0.01

hn_neurons = [100,75,50,25,12]

ac_fun     = ['relu', 'relu', 'relu', 'relu', 'linear']

ls_fun     = 'mean_squared_error'
op_val     = 'adam'
it_val     = 20
bt_size    = 512

sh_val     = True # shuffle
vr_val     = 1    # learning to be printed


In [ ]:
#@title 12.2.4. Construct the neural network

net        = tf.keras.models.Sequential() # back-to-back layers of neurons (platform)

net.add( tf.keras.layers.Dense(units=hn_neurons[0], activation=ac_fun[0], input_dim = in_neurons) ) # input layer and the 1st hidden layer
net.add( tf.keras.layers.Dense(units=hn_neurons[1], activation=ac_fun[1]) )                         # 2nd hidden layer
net.add( tf.keras.layers.Dense(units=hn_neurons[2], activation=ac_fun[2]) )                         # 3rd hidden layer
net.add( tf.keras.layers.Dense(units=hn_neurons[3], activation=ac_fun[3]) )                         # 4th hidden layer
net.add( tf.keras.layers.Dense(units=ou_neurons   , activation=ac_fun[4]) )                         # output layer

net.compile(optimizer = op_val, loss = ls_fun) # compile the network

In [ ]:
#@title 12.2.5. Network summary
net.summary()

In [ ]:
#@title 12.2.6. Train the network
history = net.fit(datain_tr_calibrated,
                  dataou_tr_calibrated,
                  epochs           = it_val,
                  batch_size       = bt_size, 
                  verbose          = vr_val,
                  shuffle          = sh_val,
                  validation_split = val_split)

In [ ]:
#@title 12.2.7. Estimate outputs of testing data
dataes_te_calibrated = net.predict(datain_te_calibrated)

In [ ]:
#@title 12.2.8. Plot testing real vs estimated values 
plt.figure(figsize=[7,7])

plt.plot(dataou_te_calibrated.ravel(), dataes_te_calibrated.ravel(), 'b*',markersize = 1)
plt.plot([-1.1,1.1], [-1.1,1.1],'r--',linewidth = 2)

plt.title('Real VS Estimated Stresses | Testing', fontsize = 20)
plt.xlabel('Real', fontsize = 20)
plt.ylabel('Estimated', fontsize = 20)
plt.legend(['Output Point','Bisector'], fontsize = 20)


In [ ]:
#@title 12.2.9. Plot iteration vs loss
plt.figure(figsize=[7,7])
plt.plot(history.history['loss'],'-')
plt.plot(history.history['val_loss'],'--')
plt.title('Model Loss', fontsize = 20)
plt.ylabel('Loss', fontsize = 20)
plt.xlabel('Iteration Number', fontsize = 20)
plt.legend(['Validation Loss', 'Testing_Loss'], fontsize = 20)